In [1]:
#Import necessary libraries
import pandas as pd

In [2]:
#Load the raw data
df = pd.read_excel("Raw_Data.xlsx")

In [3]:
#Drop records having null values
df = df.dropna()

#df.to_csv('dropna.csv')

In [4]:
print(df.shape)

(42, 3)


In [5]:
#Define response options based on questionnaire
response_options = ["Comfortable and Clean Rooms",
                    "Delicious Breakfast",
                    "Fast and Reliable Wi-Fi",
                    "Friendly and Helpful Staff",
                    "Quiet and Restful Environment",
                    "Modern Fitness Facilities",
                    "Family-Friendly Services",
                    "Business Amenities",
                    "Easy Parking & Check-in",
                    "Stylish Interior Design",
                    "Reservation & Communication"
                   ]

In [6]:
# Clean and split responses into lists
# Remove any leading and trailing whitespace from semicolon-separated strings and returns a list of responses.

def trim_responses(df_column):
    return df_column.apply(
        lambda x: [response.strip() for response in str(x).split(';')] if pd.notnull(x) else []
    )

# Apply function to both check-in and check-out, and save to a single DataFrame
cleaned_responses_df = pd.DataFrame({
    "Customer": df["Customer"],
    "Cleaned_CheckIn_Responses": trim_responses(df["Check-In Questionnaire: What persuaded you to choose our hotel for your stay?"]),
    "Cleaned_CheckOut_Responses": trim_responses(df["Check-Out Questionnaire: What impressed you the most during your stay?"])
})

In [7]:
# Function to validate cleaned responses
# Check for typos or inconsistent responses

def validate_responses(df_column, valid_options, customer_column):
    invalid_response_customers = 0
    for idx, responses in df_column.items():
        invalid = [resp for resp in responses if resp not in valid_options]
        if invalid:
            customer = customer_column[idx]
            print(f"Customer: {customer} | Invalid Responses: {invalid}")
            invalid_response_customers += 1
    print(f"\nCustomers with Invalid Responses: {invalid_response_customers}")

# Validate Check-In responses
validate_responses(
    cleaned_responses_df["Cleaned_CheckIn_Responses"],
    response_options,
    cleaned_responses_df["Customer"]
)

# Validate Check-Out responses
validate_responses(
    cleaned_responses_df["Cleaned_CheckOut_Responses"],
    response_options,
    cleaned_responses_df["Customer"]
)


Customers with Invalid Responses: 0

Customers with Invalid Responses: 0


In [8]:
# Perform one-hot encoding

# Function to convert cleaned list of responses into a semicolon-separated string (for get_dummies)
def join_responses(response_series):
    return response_series.apply(lambda x: ';'.join(x))

# Join cleaned response lists back into strings
checkin_responses = join_responses(cleaned_responses_df["Cleaned_CheckIn_Responses"])
checkout_responses = join_responses(cleaned_responses_df["Cleaned_CheckOut_Responses"])

checkin_onehot_df = checkin_responses.str.get_dummies(sep=';').reindex(columns=response_options, fill_value=0)
checkout_onehot_df = checkout_responses.str.get_dummies(sep=';').reindex(columns=response_options, fill_value=0)

In [9]:
# Aggregate (sum) the binary values
checkin_response_count = checkin_onehot_df.sum().rename("Check-In Response Count")
checkout_response_count = checkout_onehot_df.sum().rename("Check-Out Response Count")

In [10]:
# Comparison
comparison_df = pd.concat([checkin_response_count, checkout_response_count], axis=1)
comparison_df["Difference"] = comparison_df["Check-Out Response Count"] - comparison_df["Check-In Response Count"]
comparison_df.sort_values("Check-In Response Count", ascending=False, inplace=True)

print(comparison_df)

                               Check-In Response Count  \
Quiet and Restful Environment                       33   
Comfortable and Clean Rooms                         24   
Fast and Reliable Wi-Fi                             24   
Delicious Breakfast                                 23   
Modern Fitness Facilities                           22   
Business Amenities                                  16   
Friendly and Helpful Staff                          15   
Family-Friendly Services                            11   
Easy Parking & Check-in                             10   
Stylish Interior Design                             10   
Reservation & Communication                         10   

                               Check-Out Response Count  Difference  
Quiet and Restful Environment                        32          -1  
Comfortable and Clean Rooms                          10         -14  
Fast and Reliable Wi-Fi                              21          -3  
Delicious Breakfast    

In [11]:
#Save the results
comparison_df.to_csv('response_comparison.csv')